# code to import predictions for Types having 7 days predictions:



1 Wind
2 Solar
17 WindOffshore
20 Nuclear
21 WastePower
26 OtherPower
59 ElectricityLoad (as of 9 April 2025, noon UTC)



Import required dependencies

In [1]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [2]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [3]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

In [ ]:
from datetime import datetime, timedelta, time as dtime
import time
import os
import requests
import pandas as pd

n_values = [1, 2, 17, 20, 21, 26, 59]  # Selected Types of Electricity Sources (NED.nl)

FETCH_TIMES_UTC = [dtime(0, 0, 5), dtime(12, 0, 5)]  # 00:00:05 and 12:00:05 UTC

def wait_until_next_fetch():
    now = datetime.utcnow()
    future_fetch_times = [datetime.combine(now.date(), t) for t in FETCH_TIMES_UTC]

    # Als we al voorbij het laatste tijdstip van vandaag zijn, plan voor morgen
    future_fetch_times = [t if t > now else t + timedelta(days=1) for t in future_fetch_times]
    
    next_fetch_time = min(future_fetch_times)
    wait_seconds = (next_fetch_time - now).total_seconds()
    print(f"Waiting until next fetch at {next_fetch_time} UTC ({int(wait_seconds)} seconds)...")
    time.sleep(wait_seconds)

def fetch_and_save_data():
    current_datetime = datetime.utcnow()
    future_datetime = current_datetime + timedelta(days=8)

    current_date_str = current_datetime.strftime("%Y-%m-%d")
    future_date_str = future_datetime.strftime("%Y-%m-%d")

    for gen_type in n_values:
        api_params = {
            "point": "0",
            "type": gen_type,
            "granularity": "5",  # hourly
            "granularitytimezone": "1",
            "classification": "1",  # Forecast
            "activity": "1",
            "validfrom[before]": future_date_str,
            "validfrom[after]": current_date_str,
            "page": 1
        }

        full_data = []

        response = requests.get(API_URL, params=api_params, headers=api_headers)
        if response.status_code == 200:
            data = response.json()
            last_page_url = data.get("hydra:view", {}).get("hydra:last", None)
            last_page_num = int(last_page_url.split("page=")[-1]) if last_page_url else 1
        else:
            print(f"Error {response.status_code}: {response.text}")
            return

        for page in range(1, last_page_num + 1):
            api_params["page"] = page
            response = requests.get(API_URL, params=api_params, headers=api_headers)
            if response.status_code == 200:
                json_dict = response.json()
                if "hydra:member" in json_dict:
                    full_data.extend(json_dict["hydra:member"])
            else:
                print(f"Error on page {page}: {response.status_code}")
                break

        df = pd.DataFrame(full_data)
        df['current_datetime'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        # File path for teh appended CSV file
        output_dir = "../Data"
        file_path = os.path.join(output_dir, "appended_NED_preds.csv")

        # Create the directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Check if the file already exists
        if not os.path.exists(file_path):
            # If the file does not exist, write the header (first write)
            df.to_csv(file_path, index=False)
        else:
            # If the file exists, append to it (without writing the header)
            df.to_csv(file_path, mode='a', header=False, index=False)

        print(f"Data collection complete! Saved {len(df)} records of Type-{gen_type} to csv")

# Main loop
while True:
    fetch_and_save_data()
    wait_until_next_fetch()

Data collection complete! Saved 182 records of Type-1 to csv
Data collection complete! Saved 182 records of Type-2 to csv
Data collection complete! Saved 182 records of Type-17 to csv
Data collection complete! Saved 182 records of Type-20 to csv
Data collection complete! Saved 182 records of Type-21 to csv
Data collection complete! Saved 182 records of Type-26 to csv
Data collection complete! Saved 182 records of Type-59 to csv
Waiting until next fetch at 2025-04-11 00:00:05 UTC (35561 seconds)...


# to add function to regularly make API calls, also need to add a timestamp